# Machine Learning Tutorial

We will build a Deep Neural Network to classify sprinklings of various discrete spacetimes (causal sets). We will use Google's TensorFlow (https://www.tensorflow.org) to build the DNN, and then use data that I will provide to train and test the DNN.

Acknowledgements: I'd like to thank Will Cunningham for most of the data parsing code and help with much of the code in general. 

Comment: If you find that some of the steps taken in this tutorial are not clear I'd recommend you try to print the output of the statements that are unclear, so that you can see exactly what is going on. I have left a few (commented) print statements here and there which you can try to uncomment and execute should you be interested in seeing what is going on. This is especially useful when it comes to seeing how the data is structured.

## Download Data

You can find the data, together with this notebook and any other material that is relevant for the tutorial, here:

https://github.com/dbenincasa/ml-tutorial.git

To be able to run the code below you'll need to download the data on your machine and modify the 'basedir' variable to point to the folder containing the data files.

## Import relevant libraries

The first step is to install tensorflow. You can follow the instructions in the link above. Should you have any problems with the installation we can try to fix it during our discussion (with Jef's help!). 

As well as tensorflow we will need the numpy and pandas libraries. The former should already be part of your python installation, while if you don't have the latter you can install it using pip: 

$pip install pandas

In [46]:
import numpy as np
import pandas as pd
import tensorflow as tf

## Import and parse data

We begin by importing and parsing the datasets.

In [2]:
# replace the path in the basedir variable with your own local path
basedir = '/Users/dbenincasa/Documents/machine-learning/ml-tutorial/'
files = pd.read_table(basedir + 'training_files.dat', delim_whitespace = True, 
                      comment = '#', names = ['label', 'filename'])
#print(files, '\n')

tp = [pd.read_table(file, delim_whitespace = True, names = range(100), dtype = int) 
      for file in files['filename'].values]
# a table of size 10000x100. Each row corresponds to one training data point. The columns
# correspong to abundances of order intervals N0, N1, ..., N99.
#print(tp[0])

ltp = [len(p) for p in tp]
# a list of sizes of training sets for each class.
#print(ltp)

In [51]:
# Use pandas concat method to concatenate the data for the three classes together
training_predictors = pd.concat(tp).reset_index(drop=True)

# We'll may need the following should we decide to train with causets of size > 100. As is is
# it doesn't really do anything
training_predictors = training_predictors[training_predictors.columns[:100]]#[:1000:10]

Pandas has some very nice methods that allow us to easily display information about our data

In [54]:
#training_predictors.head()

In [55]:
#training_predictors.describe()

### Define the training classes

The class label for each training data must be given as a vector of dimension equal to the number of output neurons (i.e. the number of classes), with a 1 for the correct class and zero everywhere else. 

In [57]:
ser = []
total = 0
training_classes = pd.DataFrame()
for i, label in enumerate(files['label'].values):
    if not -label in training_classes.columns:
        training_classes[-label] = pd.concat([pd.Series([0 for x in range(0, total)]),
                                             pd.Series([1 for x in range(total, total + ltp[i])]),
                                             pd.Series([0 for x in range(ltp[i], len(training_predictors))])]).reset_index(drop=True)
    else:
        training_classes[-label][total:(total+ltp[i])] = 1
    total += ltp[i]

In [58]:
#training_classes.head()

Randomly shuffle data. This step is not needed if we use the full data set for training, but we will actually split the data into a training set and a validation set. The latter is usually used to tune hyperparameters. We will use it primarily to do a preliminary test of our model.

In [5]:
# Append training classes columns to end of training predictors matrix, and randomly select 100% of the data: 
# sample(frac=1).
shuffled_data = pd.concat([training_predictors, training_classes], axis=1).sample(frac=1).reset_index(drop=True)
training_predictors = shuffled_data[shuffled_data.columns[:100]]
training_classes = shuffled_data[shuffled_data.columns[-3:]]

### Data Recap

Our training data is composed of pairs (x_i, y_i), i = 1,...,10000, where the training inputs x_i, each contain 100 features corresponding to 100 types of order intervals, and the labels y_i determine which class each input corresponds to. Note that each label y_i is a "one-hot" vector, e.g. an input corresponding to d = 2 has y = (1, 0, 0). This representation of the labels is needed for tensorflow to work.

All this data has been neatly put into a pandas dataframe which is very easy to manipulate. Although we're not going to analyse the data more thoroughly in this tutorial, the pandas dataframe is very effective at doing data analysis should you wish to do so (we may see a few examples of its capabilities at a later stage).

## Tensorflow

WARNING: A new version of tensorflow is out (v1.6) which has higher-level APIs that simplify things considerably. In particular, many of the functions that we'll use in this tutorial can be entirely bypassed in the current version. TF current tutorial says "We recommend using the higher level APIs to build models when possible", but because my code was written with the older version I'll stick with the lower level APIs for now, and will probably upgrade at a later date.

In this section I will explain some of tensorflow's APIs (the ones I use) as I build the network. As such there will be many aspects of tensorflow that I won't talk about (and don't know about!) but that you can read about in the official documentation. 

The central unit of TF is the *tensor*. Tensors are specified by their *rank* (the number of dimensions), e.g. rank 0 = scalar, rank 1 = vector, rank 2 = matrix etc., and *shape*  (the length of the array along each dimension).

A computational graph is a series of TF operations arranged in a graph that is composed of two types of objects:

1) Operations -- the graph's nodes

2) Tensors -- the graph's edges. These represent the values that flow through the graph.

To evaluate tensors, instantiate a tf.Session object, informally known as a session. A session encapsulates the state of the TF runtime, and runs TF operations. If a tf.Graph is like a .py file, a tf.Session is like the python executable.

In [6]:
sess = tf.Session()

### DNN architecture:

We will define the architecture of our DNN using the following parameters:

In [7]:
N = 100 # Number of inputs (features/predictors/factors) for each causal set, i.e. interval abundances
D = 3   # Number of outputs (classes), i.e. dimensions (2,3,4)
H = [72, 48, 24]   # The number and sizes of the hidden layers. Here we have three hidden layers with 72, 48, 
                   # and 24 neurons, respectively
L = [N] + H + [D]  # All the layer sizes together

A TF graph can be parameterized to accept external inputs, such as our abundances of order intervals. These known as *placeholders*. A placeholder is a promise to provide a value later, like a function argument.

In [8]:
predictors = tf.placeholder("float", [None, N], name="predictors") # placeholder for inputs
classes = tf.placeholder("float", [None, D], name="classes") # placeholder for outputs

Weights and biases are defined as *variables* in tensorflow. Basically variables can get updated by tensorflow's operations. 

We now define two functions. 

The first takes as inputs the number of inputs and outputs of a given layer and initialises the weights and biases associated to that layer at random according to a normal distribution. E.g. our first hidden layer takes in 100 inputs and spits out 72 outputs, so the weight matrix associated to that layer is 100x72 and the bias vector is 72x1.

In [9]:
def get_wb(num_inputs, num_outputs):
        weights = tf.Variable(tf.truncated_normal([num_inputs, num_outputs], stddev = 0.0001))
        biases = tf.Variable(tf.ones([num_outputs]))
        return weights, biases

The second function returns the output of each layer given some input. We will use a rectified linear unit (ReLU) as the activation function for our hidden neurons, while the output layer will be a softmax layer.

In [10]:
def get_hl(data, weights, biases):
        layers = []
        for i in range(len(weights) - 1):
                layers.append(tf.nn.relu(tf.matmul(data, weights[i]) + biases[i]))
                data = layers[i]
        model = tf.nn.softmax(tf.matmul(layers[-1], weights[-1]) + biases[-1], name="model")
        return model, layers

In [61]:
# Initialise the weight matrix and bias for each layer:
W = []
b = []
for i in range(len(L) - 1):
        w0, b0 = get_wb(L[i], L[i+1])
        W.append(w0)
        b.append(b0)

# Check that the shape is correct:
#W

In [12]:
# Feed the above weights and biases to our network.
model, _ = get_hl(predictors, W, b)

### Choice of cost function

As our cost function we'll use cross-entropy. 

ASIDE: Since the true p.m.f. of each training example is concentrated on the correct class, e.g. p = [0,1,0], the cross entropy cost function can also be seen as the log-likelihood. Hence, minimization of cross-entropy is equivalent to MLE. Also note that minimizing cross entropy is equivalent to minimizing KL divergence.

Try experimenting with different kinds of cost functions.

In [13]:
cost = -tf.reduce_sum(classes * tf.log(tf.clip_by_value(model, 1e-10, 1.0)), name="cost")
# cost = tf.reduce_mean(-tf.reduce_sum(classes * tf.log(model), reduction_indices=[1]), name = "cost")
# cost = -tf.reduce_sum(classes * tf.square(tf.clip_by_value(model, 1e-10, 1.0)), name="cost")
# cost = tf.reduce_sum(tf.square(model - classes)) / 3.0

### Training algorithm

I've found that the adam optimizer coverges faster than standard SGD. You should still experiment with different optimizers to see what happens

The optimizer usually needs a learning rate parameter which we also define below. The adam optimizer comes with a default learning rate that works very well, but one can still feed it a different value.

In [14]:
learning_rate = 0.0001

In [15]:
op = tf.train.AdamOptimizer(learning_rate).minimize(cost)
#op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

### Accuracy

We now have all the necessary ingredients to train our network. Before we do so let us first define accuracy functions that we can evaluate during training to see how well the training process is doing as it's running.

In [16]:
correct_prediction = tf.equal(tf.argmax(model, 1), tf.argmax(classes, 1), name="correct_prediction")
# argmax returns the index with the largest value. If this is equal to the index of the correct class
# then we have a correct prediction

# accuracy is the percentage of correct predictions
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"), name="accuracy")

## Train the DNN

To train the DNN we must first initialize the tensorflow session as well as all global variables. This is done with the following commands:

In [35]:
# Initialize TF session and global variables
init = tf.global_variables_initializer()
sess.run(init)

### Training/Validation split

We will split our training data into a training set and a validation set. The validation set should be used to tune the model's hyperparameters, e.g. network architecture, learning_rate, batch size, etc. For now you can simply think of it as our test set which we use to evaluate how well our model generalises (i.e. how well it does when faced with data that it's never seen before).

In [36]:
# The first 80% of the data will be for training
# and the last 20% for testing
training_size = int(len(training_predictors) * 0.8)
test_size = len(training_predictors) - training_size

Finally, recall that to speed up training we use mini-batches, rather than the full training data, to compute the gradient. So we define the mini-batch size as well as the number of training epochs. 

In [37]:
# MINI-BATCH SIZE
B = 1000
# NUMBER OF TRAINING EPOCHS
T = 1000
# This variable determines every how many steps we evaluate the model's accuracy and cost.
prnt = 100

In [38]:
# Train the DNN
print('Epoch \t Cost \t\t Accuracy')
print('-----------------------------------')
if(B > 0):
        for i in range(1, T + 1):
            for j in range(0, training_size, B):
                    batch_predictors = training_predictors[j:(j+B)]
                    batch_classes = training_classes[j:(j+B)]
                    _, curr_model, curr_cost = sess.run([op, model, cost], 
                                                        feed_dict={predictors: batch_predictors, classes: batch_classes})
            if i % prnt == 0:
                    acc = sess.run(accuracy, feed_dict = {predictors: training_predictors[:training_size],
                                                      classes: training_classes[:training_size]})
                    print(i, curr_cost, acc)
else:
        for i in range(1, T + 1):
                _, curr_model, curr_cost = sess.run([op, model, cost],
                feed_dict={predictors: training_predictors[:training_size],
                classes: training_classes[:training_size]})
                if i % prnt == 0:
                        print(i, curr_cost, sess.run(accuracy,
                              feed_dict={predictors:training_predictors[:training_size],
                                         classes: training_classes[:training_size]}))

Epoch 	 Cost 		 Accuracy
-----------------------------------
100 253.04832 0.99916667
200 185.0933 0.99929166
300 132.28088 0.999375
400 92.91496 0.99954164
500 64.45954 0.99954164
600 44.355186 0.9995
700 30.359287 0.9994583
800 20.708824 0.9995
900 14.092503 0.9994583
1000 9.578306 0.9994583

Accuracy: 0.999


InvalidArgumentError: assertion failed: [`labels` out of bound] [Condition x < y did not hold element-wise:x (confusion_matrix/control_dependency:0) = ] [1 3 2...] [y (confusion_matrix/Cast_2:0) = ] [3]
	 [[Node: confusion_matrix/assert_less/Assert/AssertGuard/Assert = Assert[T=[DT_STRING, DT_STRING, DT_INT64, DT_STRING, DT_INT64], summarize=3, _device="/job:localhost/replica:0/task:0/device:CPU:0"](ConstantFolding/confusion_matrix/assert_less/Assert/AssertGuard/Assert/Switch-0, confusion_matrix/assert_less/Assert/AssertGuard/Assert/data_0, confusion_matrix/assert_less/Assert/AssertGuard/Assert/data_1, ConstantFolding/confusion_matrix/assert_less/Assert/AssertGuard/Assert/Switch_1-0, confusion_matrix/assert_less/Assert/AssertGuard/Assert/data_3, ConstantFolding/confusion_matrix/assert_less/Assert/AssertGuard/Assert/Switch_2-0)]]

Caused by op 'confusion_matrix/assert_less/Assert/AssertGuard/Assert', defined at:
  File "/Users/dbenincasa/anaconda3/lib/python3.6/runpy.py", line 193, in _

### Evaluate model on validation set

Having trained our model with the given hyperparameters we can now evaluate it on the validation set. The model already works very well, but normally one would use this validation set to tune the hyperparameters and then test the final model on some other, never seen, data set.

The confusion matrix is a way to efficiently analyse the performance of the network.

In [44]:
#Finally let us evaluate the performance of our model on the remaining 20% of the data
if test_size > 0:
        feed_dict = {predictors: training_predictors[-test_size:],
                     classes: training_classes[-test_size:]}
        print('\nAccuracy:', sess.run(accuracy, feed_dict))
        m = sess.run([op, tf.argmax(model, 1)], feed_dict)
        cols=training_classes.columns
        #print(cols)
        bt=training_classes[-test_size:].apply(lambda x : x > 0)
        #print(bt)
        p = bt.apply(lambda x : list(abs(cols[x.values])-2)[0], axis=1)
        #print(p)
        confusion = tf.confusion_matrix(labels=p, predictions=m[1], num_classes= D)
        with sess.as_default():
                print(confusion.eval())


Accuracy: 0.999
[[2066    0    0]
 [   0 1945    3]
 [   0    3 1983]]


# Save the model and quit the session

We can now save the model just trained so that we can use it outside of this notebook.

In [45]:
# Save everything
saver = tf.train.Saver()
# Save the model
saver.save(sess, basedir + 'causets')

'/Users/dbenincasa/Documents/machine-learning/ml-tutorial/causets'

And finally we can quit the sessio by running the following command:

In [ ]:
sess.close()

This isn't needed here but would be had we been running this code in a python script.